<img src="../Img/banner-fa-49.png">

Учебные материалы дисциплины "Программирование на языках Python и SQL" предназначены для семинарских занятий со студентами II курса Финансового университа при Правительстве Российской Федерации.

Автор: Смирнов Михаил Викторович, доцент Департамента анализа данных и машинного обучения Финансового университета при Правительстве Российской Федерации. mvsmirnov@fa.ru

Москва - 2021

При подготовке материалов учебных занятий использовались источники
- Essential SQLAlchemy: Mapping Python to Databases 2nd Edition. Jason Myers, Rick Copeland. O'Reilly Media, Inc. 2015.
- Астахова И.Ф., Мельников В.М., Толстобров А.П., Фертиков В.В. СУБД: язык SQL в примерах и задачах.—М.:ФИЗМАТЛИТ, 2009. — 168 с. — ISBN 978-5-9221-0816-4.

В ряде учебных примеров использованы данные <a href="http://insideairbnb.com/get-the-data.html">Inside Airbnb</a>

<a id=T_3></a>
# Тема 3. SQLAlchemy ORM

<a id=Ref></a>
# Оглавление

[Введение](#Intro)<br>
[3.1. Определение таблиц через классы ORM](#T_3_1)<br>
[3.2. Работа с данными](#T_3_2)<br>
[3.3. Запросы к данным](#T_3_3)<br>
[3.4. Исключения и транзакции](#T_3_4)<br>
[3.5. Отображение](#T_3_5)<br>

<img src="../Img/Label_02.png">

Семинар № 9

16 апреля 2021 года <br>
ПИ19-3, ПИ19-4 - 3 подгруппа<br>

17 апреля 2021 года <br>
ПИ19-2, ПИ19-3, ПИ19-4 - 2 подгруппа

23 апреля 2021 года <br>
ПИ19-4, ПИ19-5 - 4 подгруппа

<a id=Intro></a>
# Введение
[<= ](#Ref)||[ К оглавлению ](#Ref)||[ =>](#T_3_1)

ORM - Object-relational mapping - Объектно-реляционное отображение

SQLAlchemy ORM обеспечивает эффективный способ привязки схемы и операций базы данных к объектам данных.

В SQLAlchemy Core мы создавали контейнер метаданных, а затем объявляли объект *Table*, связанный с этими метаданными. В SQLAlchemy ORM мы будем определять класс, который наследуется от специального базового класса *declarative_base*. Этот базовый класс объединяет контейнер метаданных и средство сопоставления, которое сопоставляет наш класс с таблицей базы данных. Он также сопоставляет экземпляры класса с записями в этой таблице.

<a id=T_3_1></a>
[<= ](#Intro)||[ К оглавлению ](#Ref)||[ =>](#T_3_2)

# 3.1. Определение таблиц через классы ORM

ORM классы должны:
- Происходить от класса *declarative_base*.
- Содержать `__tablename__`, которое является именем таблицы базы данных.
- Содержать один или несколько атрибутов, которые являются объектами *Column*.
- Содержать атрибуты, составляющие первичный ключ.

Изучим требование, связанное с атрибутами. Определение столбцов в классе ORM похоже на определение столбцов в объекте *Table*, которое мы изучили в теме SQLAlchemy Core. Однако есть важное отличие. При определении столбцов в классе ORM в качестве имени столбца будет установлено имя атрибута класса, которому он назначен. Все остальное, что связано с типами данных и столбцами, применимо и здесь.

<img src="./Img/Listings_ORM_Schema.png">

<br><br>
Определим таблицу *listings* как класс ORM

In [1]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import (Table, Column, 
                        Integer, Numeric, String, Boolean,
                        ForeignKey, ForeignKeyConstraint, CheckConstraint)

from datetime import datetime
from sqlalchemy import DateTime

Base = declarative_base()

In [2]:
class Listings(Base):
    __tablename__ = 'listings'

    listing_id = Column(Integer(), primary_key = True)
    listing_name = Column(String(50), index = True, nullable = False)
    listing_url = Column(String(50))
    host_id = Column(Integer())
    neighbourhood_id = Column(Integer())
    amenities = Column(String(250))
    property_type_id = Column(Integer())
    room_type_id = Column(Integer())
    bedrooms = Column(Integer())
    beds = Column(Integer())
    price = Column('price',Numeric(7,2))
    
    __table_args__ = (
        ForeignKeyConstraint(['neighbourhood_id'],['neighbourhoods.neigh_id']),
        ForeignKeyConstraint(['property_type_id'], ['property_types.property_type_id']),
        ForeignKeyConstraint(['room_type_id'], ['room_types.room_type_id']),
        CheckConstraint('price >= 0.00', name='listing_price_positive')
    )
    

В этом примере *Base* - экземпляр класса *declarative_base()*. Затем создается дочерний класс *Listings*. Определяется имя таблицы 'listings'. Определяются атрибуты, устанавливается первичный ключ. Обратимся к свойству `__table__` класса.

In [3]:
Listings.__table__

Table('listings', MetaData(bind=None), Column('listing_id', Integer(), table=<listings>, primary_key=True, nullable=False), Column('listing_name', String(length=50), table=<listings>, nullable=False), Column('listing_url', String(length=50), table=<listings>), Column('host_id', Integer(), table=<listings>), Column('neighbourhood_id', Integer(), ForeignKey('neighbourhoods.neigh_id'), table=<listings>), Column('amenities', String(length=250), table=<listings>), Column('property_type_id', Integer(), ForeignKey('property_types.property_type_id'), table=<listings>), Column('room_type_id', Integer(), ForeignKey('room_types.room_type_id'), table=<listings>), Column('bedrooms', Integer(), table=<listings>), Column('beds', Integer(), table=<listings>), Column('price', Numeric(precision=7, scale=2), table=<listings>), schema=None)

Создадим таблицу клиентов *users*

In [4]:
from datetime import datetime
from sqlalchemy import DateTime

class Users(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key = True)
    username = Column(String(15), nullable = False, unique = True)
    email_address = Column(String(255), nullable = False)
    phone = Column(String(20), nullable = False)
    password = Column(String(25), nullable = False)
    created_on = Column(DateTime(), default = datetime.now)
    updated_on = Column(DateTime(), default = datetime.now, onupdate=datetime.now)

Здесь мы определили несколько атрибутов, которые не могут оставаться пустыми. Требуется уникальное значение *username*. Для атрибута *updated_on* мы установили текущее время по умолчанию, если время не указано. Использование *onupdate* приведет к установке текущего времени при обновлении любого атрибута записи.

## 3.1.1. Ключи, ограничения, индексы

Ранее, в разделе *Core*, мы изучили, что ключи и ограничения могут задаваться как в составе элемента *Column()* конструктора *Table*, так и в явном виде. Например, в *line_items* атрибут *order_id* является внешним ключом, тода

`ForeignKeyConstraint(['order_id'], ['order.order_id'])`. 

В ORM также существует для этого два способа, но так как конструктор *Table* здесь не используется, то применяются свойства класса. 

```
user_id = Column(Integer(), ForeignKey('users.user_id'))
```

Для задания ограничения в явном виде в классе используется `__table_args__`

```
class SomeDataClass(Base):
    __tablename__ = 'somedatatable'
    __table_args__ = (ForeignKeyConstraint(['id'], ['other_table.id']),
                      CheckConstraint(price >= 0.00', name='unit_cost_positive'))
```
В данном примере значением `__table_args__` является кортеж.

### Задание 3.1.1.1.
Создать классы *Orders, Line_items, Hosts, Neighbourhoods, Room_types, Property_types*. Создать базу данных *Listings.db*

In [5]:
# Ваш код здесь


## 3.1.2. Сохранение схемы

In [14]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///Listings.db')

Base.metadata.create_all(engine)

## 3.1.3. Связи
В *ORM* имеются некоторые различия при связывании таблиц по сравнению с *Core*. *ORM* также использует *ForeignKey* для ограничения и связывания объектов. Однако *ORM* использует директиву *relationship* чтобы предоставить свойство доступа к связанному объекту. Это добавляет некоторые накладные расходы при использовании *ORM*; однако плюсы перевешивают недостатки. В примере показано, как определить связи с помощью методов *relationship* и *backref*.
```
from sqlalchemy.orm import relationship, backref

class Orders(Base):
    __tablename__='orders'
    order_id=Column(Integer(), primary_key=True)
    user_id=Column(Integer(), ForeignKey('users.user_id'))

    user=relationship('Users', backref=backref('orders', order_by=order_id))
```
Таким образом, в классе *Orders*, устанавливается отношение «один ко многим» с классом *User*. Мы можем связать пользователя с его заказом, обратившись к свойству *user*. Это отношение также устанавливает свойство *orders* в классе *Users* через аргумент ключевого слова *backref*, которое упорядочивается по *order_id*. Директиве *relationship* требуется целевой класс для отношения, и она может дополнительно включать обратное отношение для целевого класса. *SQLAlchemy* знает, как сопоставить заданный нами *ForeignKey* с классом, который мы определили в отношении. В этом примере команда `ForeignKey(users.user_id)` сопоставляется с классом *User* через атрибут `__tablename__` пользователей и формирует связь. В строке 
```
user=relationship('Users', backref=backref('orders', order_by=order_id))
```
устанавливается связь *один ко многим*.

Также возможно установить взаимно-однозначное отношение *один к одному*. В следующем примере класс *Line_items* имеет взаимно-однозначное отношение с классом *Listings*. Аргумент ключевого слова `uselist = False` определяет его как взаимно однозначное отношение. Здесь используется более простая обратная ссылка, поскольку нам не нужно контролировать порядок.
```
class Line_items(Base):
    
    __tablename__='line_items'
    item_id=Column(Integer(), primary_key=True)
    order_id=Column(Integer(), ForeignKey('orders.order_id'))
    listing_id=Column(Integer(), ForeignKey('listings.listing_id'))
    item_start_date=Column(DateTime(), nullable=False, default=datetime.now)
    item_end_date=Column('item_end_date', DateTime(), nullable=False)
    
    rel_order=relationship("Orders", backref=backref('line_items', order_by=line_item_id))
    rel_listing=relationship("Listings", uselist=False))
```

<a id=T_3_2></a>
[<= ](#T_3_1)||[ К оглавлению ](#Ref)||[ =>](#T_3_3)
# 3.2. Работа с данными


## 3.2.1. Сеанс

Сеанс - это способ взаимодействия *ORM SQLAlchemy* с базой данных. Она "обертывает" соединение с базой данных через механизм и предоставляет карту идентификации для объектов, которые вы загружаете через сеанс или связываете с сеансом. Карта идентификации - это структура данных, подобная кешу, которая содержит уникальный список объектов, определяемый таблицей объекта и первичным ключом. Сеанс также "обертывает" транзакцию, и эта транзакция будет открыта до тех пор, пока сеанс не будет зафиксирован или не пройзойдет откат, что очень похоже на процесс, описанный в теме *Core*.

Для нового сеанса SQLAlchemy предоставляет класс *sessionmaker*, чтобы гарантировать, что сеансы могут быть созданы с одинаковыми параметрами во всем приложении. SQLAlchemy делает это путем создания класса сеанса (Session), который настроен в соответствии с аргументами, переданными в класс *sessionmaker*, который следует использовать только один раз в глобальной области действия приложения и рассматривать как параметр конфигурации. Создадим новый сеанс, связанный с базой данных SQLite в памяти:

In [15]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker # 1 

engine = create_engine('sqlite:///:memory:') # 2

Session = sessionmaker(bind=engine) # 3

session = Session() # 4

1. Импорт модуля создания сеанса *sessionmaker*
2. База данных SQLite в памяти
3. Определение класса сеанса с привязкой к механизму.
4. Создание сеанса.

Теперь у нас есть сеанс, который мы можем использовать для взаимодействия с базой данных. Хотя у сеанса есть все необходимое для подключения к базе данных, он не подключится, пока мы не дадим ему некоторые инструкции, которые этого требуют.

Дополнительно добавим несколько методов `__repr__`, чтобы упростить просмотр и воссоздание экземпляров объектов.

In [16]:
from datetime import datetime

from sqlalchemy import (Table, Column, 
                        Integer, Numeric, String, Boolean, DateTime,
                        ForeignKey, ForeignKeyConstraint, CheckConstraint)

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

Base = declarative_base()


class Users(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key = True)
    username = Column(String(15), nullable = False, unique = True)
    email_address = Column(String(255), nullable = False)
    phone = Column(String(20), nullable = False)
    password = Column(String(25), nullable = False)
    created_on = Column(DateTime(), default = datetime.now)
    updated_on = Column(DateTime(), default = datetime.now, onupdate=datetime.now)

    def __repr__(self):
        return "Users(username='{self.username}', " \
                     "email_address='{self.email_address}', " \
                     "phone='{self.phone}', " \
                     "password='{self.password}')".format(self=self)

    
class Orders(Base):

    __tablename__ = 'orders'
    order_id = Column(Integer(), primary_key = True)
    user_id = Column(Integer())
    
    __table_args__ = (ForeignKeyConstraint(['user_id'], ['users.user_id']),)
    
    rel_user=relationship("Users", backref=backref('orders', order_by=order_id))
    
    def __repr__(self):
        return "Orders(user_id='{self.user_id}')".format(self=self)
    

class Line_items(Base):
    
    __tablename__ = 'line_items'
    item_id = Column(Integer(), primary_key = True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    listing_id = Column(Integer(), ForeignKey('listings.listing_id'))
    item_start_date = Column(DateTime(), nullable = False, default = datetime.now)
    item_end_date = Column('item_end_date', DateTime(), nullable = False)
    
    rel_order=relationship("Orders", backref=backref('line_items', order_by=item_id))
    rel_listing=relationship("Listings", uselist=False)
    
    def __repr__(self):
        return "Line_items(order_id='{self.order_id}', " \
                        "listing_id='{self.listing_id}', " \
                        "item_start_date='{self.item_start_date}', " \
                        "item_end_date='{self.item_end_date}')".format(self=self)


class Hosts(Base):
    
    __tablename__ = 'hosts'
    host_id = Column(Integer(), primary_key = True)
    host_name = Column(String(50), nullable = False)
    def __repr_(self):
        return "Hosts(host_id='{self.host_name}')".format(self=self)
    
    
class Neighbourhoods(Base):
    
    __tablename__ = 'neighbourhoods'
    neigh_id = Column(Integer(), primary_key = True)
    neigh_name = Column(String(50), nullable = False, unique = True)
    def __repr_(self):
        return "Neighbourhoods(neigh_name='{self.neigh_name}')".format(self=self)
    
    
class Room_types(Base):
    
    __tablename__ = 'room_types'
    room_type_id = Column(Integer(), primary_key = True)
    room_type_name = Column(String(50), nullable = False)
    def __repr_(self):
        return "Room_types(room_type_name='{self.room_type_name}')".format(self=self)

### Задание 3.2.1.1.

Создайте класс *Property_types* для справочника типов собственности *property_types*

In [17]:
# Ваш код здесь


### Задание 3.2.1.2.
Создайте класс *Listings* с отношениями к классам *Hosts, Neighbourhoods, Room_types, Property_types* и методами `__repr__`

In [19]:
# Ваш код здесь


In [21]:
Base.metadata.create_all(engine)

## 3.2.2. Вставка данных

In [22]:
import pandas as pd

In [23]:
df = pd.read_csv('./Data/ListingsAm.csv', sep=";")
df.head(1)

,id,listing_url,name,host_id,host_name,host_is_superhost,neighbourhood_cleansed,property_type,room_type,bathrooms_text,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,20168,https://www.airbnb.com/rooms/20168,Studio with private bathroom in the centre 1,59484,Alexander,f,Centrum-Oost,Private room in townhouse,Private room,1 private bath,...,2010-03-02,2020-04-09,89,10.0,10.0,10.0,10.0,10.0,9.0,2.58


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3535 entries, 0 to 3534
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           3535 non-null   int64  
 1   listing_url                  3535 non-null   object 
 2   name                         3532 non-null   object 
 3   host_id                      3535 non-null   int64  
 4   host_name                    3535 non-null   object 
 5   host_is_superhost            3535 non-null   object 
 6   neighbourhood_cleansed       3535 non-null   object 
 7   property_type                3535 non-null   object 
 8   room_type                    3535 non-null   object 
 9   bathrooms_text               3534 non-null   object 
 10  bedrooms                     3366 non-null   float64
 11  beds                         3532 non-null   float64
 12  amenities                    3535 non-null   object 
 13  price             

Подготовим справочник районов

In [54]:
neigh_df=pd.DataFrame(df["neighbourhood_cleansed"].value_counts()).sort_index().reset_index()
neigh_df.index=range(1, len(neigh_df)+1)
neigh_dict = neigh_df["index"].to_dict()
neigh_dict

{1: 'Bijlmer-Centrum',
 2: 'Bos en Lommer',
 3: 'Buitenveldert - Zuidas',
 4: 'Centrum-Oost',
 5: 'Centrum-West',
 6: 'De Aker - Nieuw Sloten',
 7: 'De Baarsjes - Oud-West',
 8: 'De Pijp - Rivierenbuurt',
 9: 'Geuzenveld - Slotermeer',
 10: 'IJburg - Zeeburgereiland',
 11: 'Noord-Oost',
 12: 'Noord-West',
 13: 'Oostelijk Havengebied - Indische Buurt',
 14: 'Osdorp',
 15: 'Oud-Noord',
 16: 'Oud-Oost',
 17: 'Slotervaart',
 18: 'Watergraafsmeer',
 19: 'Westerpark',
 20: 'Zuid'}

In [26]:
for key, value in neigh_dict.items():
    cRec = Neighbourhoods(neigh_id = key, neigh_name = value)
    session.add(cRec)
session.commit()

In [27]:
print(cc_rec.neigh_id, cc_rec.neigh_name)

20 Zuid


### Задание 3.2.2.1.
Создайте справочники владельцев недвижимости, типов комнат и типов собственности. При создании справочника владельцев в качестве значения первичного ключа укажите значение индекса владельца из *ListingsAm.csv*

Чтобы добавить запись, содержащую значения нескольких столбцов, например первичного ключа и названия, можно в одном выражении указать несколько параметров класса таблицы:

```
cRec=ClassName(Column_1=value_1, Column_2=value_2, ... Column_n=value_n)
sesseion.add(cRec)
```

In [28]:
# Ваш код здесь


Проверка, что записано в таблицу *hosts*, сколько записей в таблице *hosts*.

In [ ]:
Hosts.__table__

In [ ]:
from sqlalchemy import select
connection = engine.connect()
sel = select([Hosts])
rp = connection.execute(sel)
rp.fetchone()

In [ ]:
from sqlalchemy.sql import func
s=select([func.count(Hosts.__table__)])
print(str(s))
rp = connection.execute(s)
rp.scalar()

### Задание 3.2.2.2.

Наполните данными таблицу *listings*

In [ ]:
# Ваш код здесь

In [49]:
df.head(1)

,id,listing_url,name,host_id,host_name,host_is_superhost,neighbourhood_cleansed,property_type,room_type,bathrooms_text,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,20168,https://www.airbnb.com/rooms/20168,Studio with private bathroom in the centre 1,59484,Alexander,f,Centrum-Oost,Private room in townhouse,Private room,1 private bath,...,2010-03-02,2020-04-09,89,10.0,10.0,10.0,10.0,10.0,9.0,2.58


## 3.2.3. Запросы

In [78]:
session.query(Neighbourhoods.neigh_id, Neighbourhoods.neigh_name).first()

(1, 'Bijlmer-Centrum')